# Welcome to the Synergy Oneliners
Created by Fredrik Tärnell - 2019-02-20<br>

This notebook contains useful oneliners for particular occations.<br>
**Don't use them without fully understanding them**.<br> Some will wipe resources away no questions asked.

Jupyter Notebooks can be found: https://github.com/frippe75/synergy-jupyter-notebooks<br>
Together with a Docker image with all dependencies inluded and a quickstart guide.


### API information

http://h17007.www1.hpe.com/docs/enterprise/servers/oneview4.1/cic-api/en/index.html <br>



### Python specific
https://hewlettpackard.github.io/python-hpOneView/index.html<br>
https://github.com/HewlettPackard/python-hpOneView<br>

<br>

## Table of content

- **[Networks](#Network)**<br>
   + [List all networks](#Network-show)<br>
   + [Create a single network](#Network-create)<br>
   + [Delete a single network](#Network-delete)<br>
   + [Create multiple networks](#Network-bulk-create)<br>
   + [Delete multiple networks](#Network-bulk-delete)<br>
   + [Delete ALL networks](#Network-delete-all)<br>
   + [Create the netop user](#Netop)<br>
- **[Server Profiles](#SP)**<br>
   + [Power On/Off all Profiles](#SP-power)<br>
   + [Delete all Profiles](#SP-delete)<br>
- **[Server Profiles Templates](#SPT)**<br>
   + [Delete all Profiles Templates](#SPT-delete)<br>
- **[Volume Templates](#VolumeTemplates)**<br>
   + [Delete all volume templates](#VolumeTemplates-deleteall)<br>
- **[Volumes](#Volumes)**<br>
   + [Delete all volumes](#Volume-deleteall)<br>
- **[Appliances](#Appliances)**<br>
   + [eFuse Composer / Image Streamer](#Appliance-efuse)<br>
   + [eFuse Compute module](#Appliance-efuse-compute)<br>
<br><br>
# First section - Simple tasks via Python



### Import the python OneView library with some additional dependencies for this playbook

In [1]:
from hpOneView.oneview_client import OneViewClient
import pandas
from IPython.display import display, Markdown
from pprint import pprint

# Simple helper function to make nicer tables using markdown and the pandas lib
# TODO: get it into helper module that gets imported above 
def my_table(source=dict(), resource="Resource", columns='name'):
    data = pandas.DataFrame.from_dict(source)
    table = data.to_html(index=False,header=True,columns=columns)
    
    display(Markdown('### ' + resource + ' in Synergy Composer at https://' + config['ip']))
    display(Markdown(table))

### Configure your environment

In [2]:
#composer_ip = "syn420-dcs.hpedemo.local"
composer_ip = "10.10.21.110"

config = {
    "api_version": "1000",
    "ip": composer_ip,
    "credentials": {
        "userName": "Administrator",
        "authLoginDomain": "local",
        "password": 'HP3!nv3nt'
    }
}
pprint(config)

{'api_version': '1000',
 'credentials': {'authLoginDomain': 'local',
                 'password': 'HP3!nv3nt',
                 'userName': 'Administrator'},
 'ip': '10.10.21.110'}


<br><br>
## Login against the Appliance / Composer instance:
create new object "ov" with the config (see above) using the JSON config dict above

In [3]:
try:
    ov = OneViewClient(config)
    print("\nLogged in against " + config['ip'] + " successfully\n")
except HPOneViewException as e:
    print("\nLogged in failed:")
    print(e.msg)


Logged in against 10.10.21.110 successfully



<br><br>
# Networking  <a name="Network"></a>
## Show existing networks  <a name="Network-show"></a>
This example uses Pandas dataframe to create the HTML table<br>
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_html.html?highlight=to_html#pandas.DataFrame.to_html


In [ ]:
poc_networks = ov.ethernet_networks.get_all()

#print(poc_networks)

if poc_networks:
    my_table(poc_networks, resource="Subnets", columns=['name','description','vlanId','uri','purpose'])


<br><br>
## Create a network  <a name="Network-create"></a>

In [ ]:
options = {
    "name": "New network",
    "vlanId": 3000,
    "ethernetNetworkType": "Tagged",
    "purpose": "General",
    "smartLink": False,
    "privateNetwork": False,
    "type" : "ethernet-networkV4",
    "connectionTemplateUri": None
}

ethernet_network = ov.ethernet_networks.create(options)
print("Created ethernet-network " + ethernet_network['name'] + "   URI: " + ethernet_network['uri'] + "\nsuccessfully.")

<br><br>
## Delete a network  <a name="Network-delete"></a>

In [ ]:
ov.ethernet_networks.delete(ethernet_network)
print("Delete the network created above: " + ethernet_network['name'])

<br><br>
## Create bulk networks <a name="Network-bulk-create"></a>

In [12]:
options_bulk = {
    "vlanIdRange": "11-20",
    "purpose": "General",
    "namePrefix": "Net",
    "smartLink": False,
    "privateNetwork": False,
    "type": "bulk-ethernet-networkV1",
    "bandwidth": {
        "maximumBandwidth": 10000,
        "typicalBandwidth": 2000
    }
}

# create the networks
ethernet_nets_bulk = ov.ethernet_networks.create_bulk(options_bulk)

# simply display whats exepected
for net in ethernet_nets_bulk: 
  print("created network: " + net['name'] + " URI: " + net['uri'])

created network: Net_11 URI: /rest/ethernet-networks/c5e06b30-38ce-4610-aba5-e87ee9be90bc
created network: Net_12 URI: /rest/ethernet-networks/5d06575a-43d6-4a94-8880-1c0bb0f518c1
created network: Net_13 URI: /rest/ethernet-networks/04587364-71bc-4123-825f-a1c63a972103
created network: Net_14 URI: /rest/ethernet-networks/1ba54684-2385-4602-a8a7-64d77d03aa2f
created network: Net_15 URI: /rest/ethernet-networks/a247beb9-218f-43c9-8704-e8352564aad9
created network: Net_16 URI: /rest/ethernet-networks/9a696465-363a-4c36-b169-d58b8e7d2b00
created network: Net_17 URI: /rest/ethernet-networks/aa27fcef-1fc3-4d9c-aa16-89eaaa9c59c2
created network: Net_18 URI: /rest/ethernet-networks/98803094-1ddf-4b39-98ee-bc47b79fc444
created network: Net_19 URI: /rest/ethernet-networks/374ec04d-822e-4fbb-9cb0-d1bfe73660fb
created network: Net_20 URI: /rest/ethernet-networks/0af1a292-3e3a-4153-9663-065f70e7fa51


<br><br>
## Delete the bulks network above  <a name="Network-bulk-delete"></a>

In [5]:
for net in ethernet_nets_bulk:
    ov.ethernet_networks.delete(net)
    print("delete network: " + net['name'])

NameError: name 'ethernet_nets_bulk' is not defined

<br><br>
## Delete ALL networks in Oneview  <a name="Network-delete-all"></a>
This will **delete ALL networks off all types** so don't run unless you mean it!

In [11]:
all_nets = ov.ethernet_networks.get_all()
#pprint(all_nets)

for net in all_nets:
    # If the ov command below is commented out it's safe to give a try as a dry-run
    ov.ethernet_networks.delete(net)
    print("Deleting network: " + net['name'])

<br><br>
## Create the netop user which was removed part of 4.2 release  <a name="Netop"></a>


In [56]:

ics = ov.interconnects.get_all()
for ic in ics:
    if ic['model'] == "Virtual Connect SE 40Gb F8 Module for Synergy":
        print("Going to add the netop user on the interconnect with uri " + ic['uri'])
        #print(ic['uri'])
        updated_interconnect = ov.interconnects.patch(ic['uri'],
                                                        'replace',
                                                        '/netOpPasswd',
                                                        'netoppwd')
    

Going to add the netop user on the interconnect with uri /rest/interconnects/2f0f6513-9602-42cf-acc8-9832e7aaf437
Going to add the netop user on the interconnect with uri /rest/interconnects/b2d4c3bb-aadc-408b-be98-1023bc2563de


<br><br>
## Server Profile Templates <a name="SPT"></a>

### Delete ALL existing Server Profile Templates <a name="SPT-delete"></a>

WARNING: Will delete ALL profiles templates unconditionally!

In [ ]:
# get a list of all the spts
spts = ov.server_profile_templates.get_all()

# delete them one-by-one
for spt in spts: 
  print("Deleting the Server Profile Template: " + spt['name'] )
  ov.server_profile_templates.delete(spt)


<br><br>
## Server Profiles <a name="SP"></a>

### Delete ALL existing Server Profiles <a name="SP-delete"></a>

WARNING: Will delete ALL profiles unconditionally!

In [ ]:
# get a list of all the sp's
sps = ov.server_profiles.get_all()

# delete them one-by-one
for sp in sps: 
  print("Deleting the Server Profile: " + sp['name'] )
  ov.server_profiles.delete(sp)


### Power on/off  ALL existing Server Profiles <a name="SP-power"></a>

**Warning:** Will power on or off all profiles depending on the new_state variable

In [ ]:
# what powerstate to achieve
new_state = "Off" # it's either "On" or "Off"

# get a list of all the sp's
sps = ov.server_profiles.get_all()

# same config for all hardware
configuration = {
        "powerState": new_state,
        "powerControl": "MomentaryPress"
    }

# power them OFF them one-by-one
for sp in sps: 
  # print them nicely
  # pprint(sp)
  sp_uri = sp['serverHardwareUri']
  sp_name = sp['name']
  # remember some profiles be unassigned. i.e they dont have a hardware uri
  if sp_uri is not None:
      print("Powering OFF the Server Profile: " + sp_name + " using ServerHardwareUri " +  sp_uri ) 
      try:
          ov.server_hardware.update_power_state(configuration,  sp_uri)
          print("Successfully changed the power state of '" + sp_name + "' to " + configuration['powerState'])
      except HPOneViewException as e:
          print(e.msg)
  else:
      print("Server Profile: " + sp_name + " is unassigned")
      

<br><br>
## Volumes <a name="Volumes"></a>

### Delete all Volumes <a name="Volume-deleteall"></a><br>
**WARNING**: This really deletes all volumes, actual command commented out.

In [ ]:
# get a list of all the storage templates
volumes = ov.volumes.get_all()
# pprint(volumes)

# delete them one-by-one
for volume in volumes: 
  print("Deleting the Volume: " + volume['name'] )
  # ov.volumes.delete(volume)

<br><br>
## Volume Templates <a name="VolumeTemplate"></a>

### Delete all Volume Templates <a name="VolumeTemplate-deleteall"></a><br>

In [ ]:
# get a list of all the storage templates
vol_temps = ov.storage_volume_templates.get_all()

# delete them one-by-one
for vol_temp in vol_temps: 
  print("Deleting the Volume Template: " + vol_temp['name'] )
  # ov.storage_volume_templates.delete(vol_temp)

<br><br>
## Logical Interconnect Groups <a name="LIG"></a>

### Add a network to an existing LIGs uplinkset <a name="LIG-uplink-network"></a><br>

In [ ]:
lig_name = "VC-LIG01"
lig_uplink_name = "Test"
eth_name = "VLAN400_10.2"

# Get the network
eth = ov.ethernet_networks.get_by('name', eth_name)[0]

# Get Logical Interconnect Group by property
lig = ov.logical_interconnect_groups.get_by('name', lig_name)[0]

lig_uplinksets = lig['uplinkSets']
for uplinkset in lig_uplinksets:
    #pprint(uplinkset)
    if uplinkset['name'] == lig_uplink_name:
        uplinkset['networkUris'].append(eth['uri'])
        ov.logical_interconnect_groups.update(lig)
    else:
        print("Could not find the uplinkset (by_name)")     

"""   
# Get an uplink set resource by name
print("\nGet uplink set by name")
uplink_set = ov.uplink_sets.get_by('name', 'Test')
pprint(uplink_set)

# Add an ethernet network to the uplink set
# To run this example you must define an ethernet network uri or ID below
ethernet_network_id = None
if ethernet_network_id:
    print("\nAdd an ethernet network to the uplink set")
    uplink_set = ov.uplink_sets.add_ethernet_networks(created_uplink_set['uri'], ethernet_network_id)
    print("The uplink set with name = '{name}' have now the networkUris:\n {networkUris}".format(**uplink_set))

"""

<br><br>
## Appliances <a name="Appliances"></a>

### eFuse an Appliance in bay 1/2 example <a name="Appliance-efuse"></a><br>
Table from 4.1 API spec on how to target different resources within the enclosure.<br>
http://h17007.www1.hpe.com/docs/enterprise/servers/oneview4.1/cic-api/en/index.html#rest/enclosures
<img src="images/EnclosurePatchTable.PNG" height="600" width="800" align="left">

In [ ]:
# WARNINIG: Make sure you target the correct enclosure as well as the operation you want! Pay attention to the table above
#           The actual patch request is commented out below for safety.
#           This is not something you normally do, only on request from support

enc_name = "HPEDemo-POCframe01"

# Get the Enclosure
efuse_enc = ov.enclosures.get_by('name', enc_name)

# If needed pretty print the entire JSON 
#pprint (efuse_enc)

# Grab the first enclosure uri
enc_uri = efuse_enc[0]['uri']

# Setup your patch request
patch_op = "replace"
#patch_path = "/applianceBays/1/bayPowerState"  # In a single POC frame this would be the Composer i.e Bay1
patch_path = "/applianceBays/2/bayPowerState"   # In a single POC frame this would be the Image Streamer i.e Bay2
patch_value = "E-Fuse" # Not the same for all paths

print("Request a PATCH on resource " + enc_uri + " using " + "op=" + patch_op + " path=" + patch_path + " val=" + patch_value)
#patched_end = ov.enclosures.patch(enc_uri, patch_op, patch_path, patch_value)


<br><br>
### eFuse a compute module in bay6 in a particular frame <a name="Appliance-efuse-compute"></a><br>

In [ ]:
# WARNINIG: Make sure you target the correct enclosure as well as the operation you want! Pay attention to the table above
#           The actual patch is commented out below for safety.
#           This is not something you normally do, only on request from support

enc_name = "HPEDemo-POCframe01"

# Get the Enclosure
efuse_enc = ov.enclosures.get_by('name', enc_name)

# If needed pretty print the entire JSON 
#pprint (efuse_enc)

# Grab the first enclosure uri
enc_uri = efuse_enc[0]['uri']

# Setup your patch request
patch_op = "replace"
patch_path = "/deviceBays/6/bayPowerState"   # In a single POC frame this would be the Computemodule in Bay6
patch_value = "E-Fuse" # Not the same for all paths

print("Request a PATCH on resource " + enc_uri + " using " + "op=" + patch_op + " path=" + patch_path + " val=" + patch_value)
#patched_end = ov.enclosures.patch(enc_uri, patch_op, patch_path, patch_value)